In [ ]:
from pandas_lib.std_imports import *
from Server_Cleaning.modules.Assignment_datasets import *
path = '../'

# Introduction
In this document, we want to perform analyses that provide information about football clubs. 
We will pay particular attention to the average goals scored/conceded per match.

In [ ]:
# Getting the DataFrames
club_games = get_club_games(path)
games = get_games(path)

# Generation of a correlation matrix
